# Step 1. Input the required libraries

In [ ]:
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


# Step 2. Load the dataset and normalize every image

## Load the dataset 

In [ ]:
data=np.load('../input/face-image-database/ORL_faces.npz')
data


In [ ]:
data_img= pd.DataFrame.from_dict({item: data[item] for item in data.files}, orient='index')
data_img

## Normalize every image in the dataset

In [ ]:
# Normalizing training data
X_train = np.array(data['trainX'],dtype='float32')/255
X_train

In [ ]:
X_train.shape

In [ ]:
# Normalizing testing data
X_test = np.array(data['testX'],dtype='float32')/255
X_test

In [ ]:
X_test.shape

In [ ]:
# Load image labels

In [ ]:
y_train= np.array(data['trainY'])
y_test= np.array(data['testY'])

# Step 3. Split the dataset

In [ ]:
print('Training dataset X_train, has', X_train.shape[0], 'rows and', X_train.shape[1], 'columns')
print('Testing dataset X_test, has', X_test.shape[0], 'rows and', X_test.shape[1], 'columns')

In [ ]:
# Step 4. Transform the images to equal sizes to feed in CNN

In [ ]:
img_rows=112
img_cols=92
batch_size=512
img_shape=(img_rows, img_cols, 1)

In [ ]:
## change the size of images

In [ ]:
X_train.shape

In [ ]:
X_train.dtype

In [ ]:
X_test.shape

In [ ]:
print('Training dataset X_train, has', X_train.shape[0], 'and', X_train.shape[1], 'columns')
print('Testing dataset X_test, has', X_test.shape[0], 'and', X_test.shape[1], 'columns')

In [ ]:
# Reshaping X_train and X_test dataset

In [ ]:
X_train = X_train.reshape(X_train.shape[0], *img_shape)
X_test = X_test.reshape(X_test.shape[0], *img_shape)

In [ ]:
print('After reshaping, training dataset X_train, has the dimensions', X_train.shape)
print('After reshaping, testing dataset X_test, has the dimensions', X_test.shape)

##  Converting class vectors to binary class matrices for y_train and y_test

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, 40)
y_test = tf.keras.utils.to_categorical(y_test, 40)

In [ ]:
print('After converting class vectors to binary class matrices, label dataset y_train, has', y_train.shape[0], 'rows and', y_train.shape[1], 'columns')
print('After converting class vectors to binary class matrices, label dataset y_test, has', y_test.shape[0], 'rows and', y_test.shape[1], 'columns')

# Step 5. Build a CNN model that has 3 main layers:
## i. Convolutional Layer
## ii. Pooling Layer
## iii. Fully Connected Layer

In [ ]:
# Network training parameters
EPOCHS = 50
BATCH_SIZE = 512
VERBOSE = 1
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001)
VALIDATION_SPLIT = 0.95
lr=0.001

INPUT_SHAPE = img_shape
NB_CLASSES = 40 # number of outputs = number of digits

In [ ]:
# First convolution layer
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(36, (5,5), activation='relu', input_shape = INPUT_SHAPE, name="ConvLay1"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

# Second convolution layer
model.add(tf.keras.layers.Conv2D(54, (5,5), activation='relu', input_shape = INPUT_SHAPE, name="ConvLay2"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

# Flatten the layers
model.add(tf.keras.layers.Flatten())

# Feed to Dense network
model.add(tf.keras.layers.Dense(units = 1024, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 512, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 256, activation = 'relu'))

# a softmax classifier
model.add(tf.keras.layers.Dense(units = NB_CLASSES, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer=OPTIMIZER,metrics=['accuracy'], loss='categorical_crossentropy')

In [ ]:
# Fit the model
history = model.fit(np.array(X_train), np.array(y_train),
                    batch_size = BATCH_SIZE,
                    epochs = EPOCHS,
                    verbose = VERBOSE,
                    validation_data=(np.array(X_test),np.array(y_test)))

# Step 6. Train the model

In [ ]:
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print('\nTest loss:', score[0])
print('\nTest accuracy:', score[1])

# Step 7. Plot the result

In [ ]:
plt.title('Model accuracy')
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['test', 'train'], loc='lower right')
plt.show()

plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['test', 'train'], loc='upper right')
plt.show()